Notebook for working on IBM datascience capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Week 2 Segmenting and Clustering Neighborhoods in Toronto

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Testing beautiful soup code
wikilink = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
html = bs4.BeautifulSoup(wikilink, "html.parser")
for text in html.select('table'):
    print(text.getText())

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Ignoring cells with a borough that is Not assigned

In [5]:
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)

In [6]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
#Mark the rows where neighborhood value is not assigned
df['Neighborhood'] = np.where(df['Neighborhood'] == "Not Assigned", df['Borough'], df['Neighborhood'])
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#Checking the resulting dataframe

In [9]:
column_names = ["Postal Code", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df[df["Postal Code"]==postcode], ignore_index=True)
test_df

,Postal Code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [10]:
df.shape

(103, 3)

In [11]:
coords = pd.read_csv('D:\IBM_DataScience\Capstone\Capstone-Project\Week3\Geospatial_Coordinates.csv')

In [12]:
#coords.rename(columns={"Postal Code": "Postal code"}, inplace=True)
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging data from wiki with latitude and longitude data from csv

In [14]:
df_coords = df.merge(coords, on="Postal Code", how="left")
df_coords.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Checking if the resultant df is same as in problem

In [15]:
column_names = ["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_coords[df_coords["Postal Code"]==postcode], ignore_index=True)
    
test_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [16]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize

C:\Users\Poonam\Anaconda2\lib\site-packages\folium\__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


#Getting the location of Toronto using geopy

In [24]:
address = 'Toronto'

geoloc = Nominatim(user_agent="capstone")
try:
    loc = geolocator.geocode(address, timeout=10)
    latitude = loc.latitude
    longitude = loc.longitude
    print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(my_address, e.message))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coords['Latitude'], df_coords['Longitude'], df_coords['Borough'], df_coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [27]:
# filter borough names that contain the word Toronto
borough_names = list(df_coords.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [28]:
df_coords_toronto = df_coords[df_coords['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_coords_toronto.shape)
df_coords_toronto.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [29]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coords_toronto['Latitude'], df_coords_toronto['Longitude'], df_coords_toronto['Borough'], df_coords_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [70]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


In [ ]:
Getting venues

In [34]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_coords_toronto['Latitude'], df_coords_toronto['Longitude'], df_coords_toronto['Postal Code'], df_coords_toronto['Borough'], df_coords_toronto['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [37]:
# Venues list added to a dataframe
venues = pd.DataFrame(venues)

venues.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues.shape)
venues.head()

(1624, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [38]:
print('{} venues were returned by Foursquare.'.format(venues.shape[0]))

1624 venues were returned by Foursquare.


In [43]:

# one hot encoding
toronto_onehot = pd.get_dummies(venues[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = venues['Postal Code'] 
toronto_onehot['Borough'] = venues['Borough'] 
toronto_onehot['Neighborhoods'] = venues['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1624, 236)


,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
toronto_grouped = toronto_onehot.groupby(["Postal Code", "Borough", "Neighborhoods"]).mean().reset_index()
toronto_grouped

,Postal Code,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478
7,M4S,Central Toronto,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.026316,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000


In [53]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal Code', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 8)


,Postal Code,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Neighborhood,Trail,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore
2,M4L,East Toronto,"India Bazaar, The Beaches West",Fast Food Restaurant,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,Sandwich Place,Breakfast Spot,Hotel,Food & Drink Shop,Department Store
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Miscellaneous Shop,Seafood Restaurant
7,M4S,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Sushi Restaurant,Supermarket,Vietnamese Restaurant


In [55]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Postal Code", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 0, 3, 3, 3, 1, 3])

In [59]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df_coords_toronto.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("Postal Code"), on="Postal Code")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 11)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Pub,Bakery,Theater
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Bank,Beer Bar,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Clothing Store,Coffee Shop,Café,Restaurant,Cosmetics Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center


In [60]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 11)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Gym,Hotel
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,2,Thai Restaurant,Café,Mexican Restaurant,Fast Food Restaurant,Bakery
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Pub,Bakery,Theater
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Café,Sandwich Place,Coffee Shop,Park,Pub
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,3,Gift Shop,Bookstore,Restaurant,Bar,Movie Theater
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,3,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,3,Pizza Place,Café,Coffee Shop,Pub,Italian Restaurant
29,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store


In [64]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [65]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,East Toronto,0,Health Food Store,Pub,Neighborhood,Trail,Distribution Center


In [66]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Hotel


In [67]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,West Toronto,2,Thai Restaurant,Café,Mexican Restaurant,Fast Food Restaurant,Bakery


In [68]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Park,Pub,Bakery,Theater
24,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Park,Pub
25,West Toronto,3,Gift Shop,Bookstore,Restaurant,Bar,Movie Theater
26,Central Toronto,3,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Café
27,Downtown Toronto,3,Café,Japanese Restaurant,Bar,Italian Restaurant,Bookstore
28,West Toronto,3,Pizza Place,Café,Coffee Shop,Pub,Italian Restaurant
29,Central Toronto,3,Playground,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store
30,Downtown Toronto,3,Café,Coffee Shop,Vietnamese Restaurant,Bakery,Mexican Restaurant
31,Central Toronto,3,Coffee Shop,Pub,Sushi Restaurant,Supermarket,Vietnamese Restaurant
32,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina


In [69]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,Central Toronto,4,Clothing Store,Coffee Shop,Yoga Studio,Miscellaneous Shop,Seafood Restaurant
10,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Brewery


Most of the neighborhoods fall under Cluster 4 which comprises mostly of cafe and restaurants.